Dependencias

In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

Se obtienen las variables numéricas y categóricas

In [55]:
df = pd.read_csv("svmynn/train.csv")
df.shape

cats = pd.DataFrame()
nums = pd.DataFrame()
for col in df.columns:
    if (df[col].min() == 0.0) and (df[col].max() == 1.0):
        cats[col] = df[col]
    else:
        nums[col] = df[col]

print("Variables categóricas:")
for col in cats.columns:
    print("La frequencia de la columna " + col + " es de: \n" + str(df[col].value_counts()))

print("Variables numéricas:")
for col in nums.columns:
    print("Los datos estadísticos de la columna " + col + " son: ")
    print(df[col].describe())

Variables categóricas:
La frequencia de la columna blue es de: 
0    1010
1     990
Name: blue, dtype: int64
La frequencia de la columna dual_sim es de: 
1    1019
0     981
Name: dual_sim, dtype: int64
La frequencia de la columna four_g es de: 
1    1043
0     957
Name: four_g, dtype: int64
La frequencia de la columna three_g es de: 
1    1523
0     477
Name: three_g, dtype: int64
La frequencia de la columna touch_screen es de: 
1    1006
0     994
Name: touch_screen, dtype: int64
La frequencia de la columna wifi es de: 
1    1014
0     986
Name: wifi, dtype: int64
Variables numéricas:
Los datos estadísticos de la columna battery_power son: 
count    2000.000000
mean     1238.518500
std       439.418206
min       501.000000
25%       851.750000
50%      1226.000000
75%      1615.250000
max      1998.000000
Name: battery_power, dtype: float64
Los datos estadísticos de la columna clock_speed son: 
count    2000.000000
mean        1.522250
std         0.816004
min         0.500000
25%   

Se realiza el cálculo de la matriz de correlación y se listan las variables con relación positiva, neutra y negativa


In [56]:
print("Matriz de correlación")
matriz = nums.corr(method="spearman")
print(matriz)

positivos = []
neutros = []
negativos = []
for index, row in matriz.iterrows():
    if (index != "price_range"):
        if (row["price_range"] > 0):
            positivos.append(index)
        if (row["price_range"] == 0):
            neutros.append(index)
        if (row["price_range"] < 0):
            negativos.append(index)

print("\nLas variables con correlación positiva respecto al rango de precio son:")
print(*positivos, sep="\n")
print("\nLas variables con correlación neutra respecto al rango de precio son:")
print(*neutros, sep="\n")
print("\nLas variables con correlación negativa respecto al rango de precio son:")
print(*negativos, sep="\n")

Matriz de correlación
               battery_power  clock_speed        fc  int_memory     m_dep  \
battery_power       1.000000     0.009161  0.034931   -0.003748  0.033412   
clock_speed         0.009161     1.000000 -0.005288    0.005447 -0.014712   
fc                  0.034931    -0.005288  1.000000   -0.027282  0.012780   
int_memory         -0.003748     0.005447 -0.027282    1.000000  0.007380   
m_dep               0.033412    -0.014712  0.012780    0.007380  1.000000   
mobile_wt           0.001752     0.010773  0.027134   -0.034259  0.022438   
n_cores            -0.029800    -0.008159 -0.015530   -0.028375 -0.004791   
pc                  0.030757    -0.005925  0.659161   -0.033373  0.027605   
px_height           0.009490    -0.013043 -0.020919   -0.001568  0.026156   
px_width           -0.009040    -0.008619 -0.009170   -0.008511  0.023180   
ram                -0.001285     0.004119  0.019897    0.033061 -0.010398   
sc_h               -0.029283    -0.030092 -0.009578   

Se realiza la clasificación mediante máquinas de vectores de soporte

In [57]:
y = df["price_range"].values
x_data = df.drop(["price_range"], axis = 1)
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

#y_train, y_test, x_train, x_test = train_test_split(df.index, df, test_size=0.30)
clasificador_svm = svm.SVC(kernel="linear", probability=True, random_state=1)
clasificador_svm.fit(x_train, y_train)

y_pred = clasificador_svm.predict_proba(x_test)
#ROC AUC tema 3.4
print("Exactitud de prueba usando ROC: ",metrics.roc_auc_score(y_test, y_pred, multi_class="ovo"))

y_pred = clasificador_svm.predict_proba(x_train)
print("Exactitud de entrenamiento usando ROC: ",metrics.roc_auc_score(y_train, y_pred, multi_class="ovo"))

Exactitud de prueba usando ROC:  0.9947844112609058
Exactitud de entrenamiento usando ROC:  0.9978985967071764


Se realiza la clasificación mediante redes neuronales

In [58]:
ohe = OneHotEncoder()
sc = StandardScaler()
x_train = np.asarray(x_train)
x_train = sc.fit_transform(x_train)
y_train = ohe.fit_transform(y_train.reshape(-1,1)).toarray()
x_test = np.asarray(x_test)
x_test = sc.fit_transform(x_test)
y_test = ohe.fit_transform(y_test.reshape(-1,1)).toarray()

clasificador_nn = Sequential()
clasificador_nn.add(Dense(16, input_dim=20, activation="relu"))
clasificador_nn.add(Dense(12, activation="relu"))
clasificador_nn.add(Dense(4, activation="sigmoid"))

clasificador_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")
clasificador_nn.fit(x_train, y_train, epochs=100, batch_size=64)

y_pred = clasificador_nn.predict(x_test)
print("Exactitud de prueba usando ROC: ",metrics.roc_auc_score(y_test, y_pred, multi_class="ovo"))

Epoch 1/100
25/25 [==============================] - 0s 760us/step - loss: 0.6848 - accuracy: 0.2788
Epoch 2/100
25/25 [==============================] - 0s 750us/step - loss: 0.6180 - accuracy: 0.2950
Epoch 3/100
25/25 [==============================] - 0s 776us/step - loss: 0.5803 - accuracy: 0.3275
Epoch 4/100
25/25 [==============================] - 0s 760us/step - loss: 0.5563 - accuracy: 0.3631
Epoch 5/100
25/25 [==============================] - 0s 872us/step - loss: 0.5367 - accuracy: 0.3994
Epoch 6/100
25/25 [==============================] - 0s 977us/step - loss: 0.5168 - accuracy: 0.4369